<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 38px; color: black; font-weight: bold;">
Latent Neural SDE VAE
</div>

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Biblio
</div>

Neural ODEs:

**Neural ODEs (https://arxiv.org/abs/1806.07366) (2019)** : introduction of the Neural ODE as the continuous-time limit of a ResNet stack. Presentation of the use of the adjoint sensitivity method. Seminal paper for Neural ODE.

**Latent ODEs for Irregularly-Sampled Time Series (https://arxiv.org/abs/1907.03907) (2019)** : Evolution of the Neural ODE model towards a Neural ODE RNN model, where the approximate posterior is built with a RNN on past observations.

Neural SDEs:

**SDE Matching: Scalable and Simulation-Free Training of Latent Stochastic Differential Equations (https://arxiv.org/abs/2502.02472 , 2025)** : good background section (#2) to explain Neural SDE. Propose a new method SDE matching, inspired by score and flow matching, vs the adjoint sensivity method. SDE matching is claimed to be more efficient to compute gradients and train latent SDEs.

**Scalable Gradients for Stochastic Differential Equations (https://arxiv.org/abs/2001.01328) (2020)** : generalization of the adjoint sensitivity method to SDEs. Combination with gradient-based stochastic variational inference for infinite-dimension VAEs.

**Neural SDEs (https://www.researchgate.net/publication/333418188_Neural_Stochastic_Differential_Equations) (2019)** : link between infinitely deep residual networks and solutions to stochastic differential equations

**Stable Neural SDEs in analyzing irregular time series data (https://arxiv.org/abs/2402.14989) (2025)** : points to the necessity of careful design of the drift and diffusion neural nets in latent SDEs. Introduces three latent SDEs models with performance guarantees.

**Generative Modeling of Neural Dynamics via Latent Stochastic Differential Equations (https://arxiv.org/abs/2412.12112) (2024)** : application of neural SDEs to a biological use case (brain activity). Details the model, architecture, ELBO/loss computation. Takes into account inputs/commands in the model. 

General/Misc:

**Efﬁcient gradient computation for dynamical models (https://www.fil.ion.ucl.ac.uk/~wpenny/publications/efficient_revised.pdf) (2014)** : summary of finite difference method, forward sensitivity method, adjoint sensitivity method, to compute gradients of a functional cost function. Applies to Neural ODEs training.

**Cyclical Annealing Schedule: A Simple Approach to Mitigating KL Vanishing (https://arxiv.org/abs/1903.10145) (2019)** : explanation of the posterior collapse/KL vanishing problem, introduces different KL annealing schedules for VAE training.


<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Code : torchsde library by Google Research
</div>

https://github.com/google-research/torchsde

[1] Xuechen Li, Ting-Kam Leonard Wong, Ricky T. Q. Chen, David Duvenaud. "Scalable Gradients for Stochastic Differential Equations". International Conference on Artificial Intelligence and Statistics. 2020. [arXiv]

[2] Patrick Kidger, James Foster, Xuechen Li, Harald Oberhauser, Terry Lyons. "Neural SDEs as Infinite-Dimensional GANs". International Conference on Machine Learning 2021. [arXiv]

[3] Patrick Kidger, James Foster, Xuechen Li, Terry Lyons. "Efficient and Accurate Gradients for Neural SDEs". 2021. [arXiv]

[4] Patrick Kidger, James Morrill, James Foster, Terry Lyons, "Neural Controlled Differential Equations for Irregular Time Series". Neural Information Processing Systems 2020. [arXiv]



<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Model & Math
</div>

Data : $\mathbf{X} = (x_{t_1}, x_{t_2}, ..., x_{t_N}) \in \mathbb{R}^{D_x}$ - assuming all $t_i \in [0,1]$.

The latent space has dimension $D_z$. The latent continuous dynamic is $\mathbf{Z}$ defined by:
\begin{align*}
z_0^{(\theta)} &\sim p_{\theta_z}(z_0) \\
dz_t^{(\theta)} &= f_{\theta}(z_t, t)dt + \sigma_{\theta}(z_t,t)dB_t 
\end{align*}
with: 
\begin{align}
\textbf{drift} \,& f_{\theta} : \mathbb{R}^{D_z} \times [0,1] \rightarrow \mathbb{R}^{D_z} \\
\textbf{diffusion} \,& \sigma_{\theta} : \mathbb{R}^{D_z} \times [0,1] \rightarrow \mathbb{R}^{D_z \times D_z} \\
\textbf{Brownian motion} \,& dB_t \in \mathbb{R}^{D_z}
\end{align}

The decoder is classically:
\begin{align}
p_{\theta_x}(x_{t_i} \vert z_{t_i})
\end{align}

The approximate posterior (encoder) is also a SDE:
\begin{align}
z_0^{(\phi)} &\sim q_{\phi}(z_0 \vert \textbf{X}) \\
dz_t^{(\phi)} &= f_{\phi}(z_t, t, \textbf{X})dt + \sigma_{\theta}(z_t,t)dB_t 
\end{align}

where:
- the drift $f_{\phi}(z_t, t, \textbf{X})$ is conditionned on observations $\textbf{X}$
- the diffusion of the approximate posterior is shared with the diffusion of the prior : $\sigma_{\theta}(z_t,t)$ - this ensures the application of Girsanov theorem and a finite KL divergence between the two stochastic processes (prior and approximate posterior) (see Generative Modeling of Neural Dynamics via Latent Stochastic Differential Equations (https://arxiv.org/abs/2412.12112) (2024))
- drift and diffusion neural nets do not exhibit the same convergence guarantee (Stable Neural SDEs in analyzing irregular time series data (https://arxiv.org/abs/2402.14989) (2025))
- non-diagonal diffusion seems to be difficult to simulate and costly to approximate (Scalable Gradients for Stochastic Differential Equations (https://arxiv.org/abs/2001.01328) (2020))
- it seems a good practice to encode only part of the $\textbf{X}$ in the approximate posterior : context vector (Scalable Gradients for Stochastic Differential Equations (https://arxiv.org/abs/2001.01328) (2020)), and $t_c << t_n$ in Generative Modeling of Neural Dynamics via Latent Stochastic Differential Equations (https://arxiv.org/abs/2412.12112) (2024).

Variational lower bound on the log marginal likelihood:

We write:
\begin{align}
p(x_{t_1:t_N}) &= \frac{p(x_{t_1:t_N}, z_{t_1:t_N})}{p(z_{t_1:t_N} \vert x_{t_1:t_N})}
\end{align}
And:
\begin{align}
\log{p(x_{t_1:t_N})} &= \int q_{\phi}(z \vert X) \log{\frac{p(x_{t_1:t_N}, z_{t_1:t_N})}{q_{\phi}(z\vert X)}\frac{q_{\phi}(z\vert X)}{p(z_{t_1:t_N} \vert x_{t_1:t_N})}} dz
\end{align}
where $q_{\phi}(z \vert X)$ is formally is posterior distribution over **functions** $z : \mathbb{R} \rightarrow \mathbb{R}^{D_z}$.
Then:
\begin{align}
\log{p(x_{t_1:t_N})} &= \int q_{\phi}(z \vert X) \log{\frac{p(x_{t_1:t_N}, z_{t_1:t_N})}{q_{\phi}(z\vert X)}} dz + \mathbb{KL}(q_{\phi}(z\vert X) \vert\vert p(z_{t_1:t_N} \vert x_{t_1:t_N}))
\end{align}
where we -audaciously- consider $p(z_{t_1:t_N} \vert x_{t_1:t_N})$ as a dsitribution over functions $z$ taking values $z_{t_1:t_N}$ at times $t_1:t_N$ so the $\mathbb{KL}$ actually means something.
Still on the same path:
\begin{align}
\log{p(x_{t_1:t_N})} &\geq \int q_{\phi}(z \vert X) \log{\frac{p(x_{t_1:t_N}, z_{t_1:t_N})}{q_{\phi}(z\vert X)}} dz \\
&= \int q_{\phi}(z \vert X) \log{\frac{p(x_{t_1:t_N} \vert z_{t_1:t_N})}{q_{\phi}(z\vert X)} p(z_{t_1:t_N})} dz \\
&= \mathbb{E}_{q_{\phi}(z \vert X)} \log{p(x_{t_1:t_N} \vert z_{t_1:t_N})} - \mathbb{KL}(q_{\phi}(z\vert X) \vert\vert p(z_{t_1:t_N})) \\
\end{align}
We write -still audaciously-
\begin{align}
\mathbb{KL}(q_{\phi}(z\vert X) \vert\vert p(z_{t_1:t_N})) &= \mathbb{KL}(q_{\phi}(z_0\vert X) \vert\vert p_{\theta_z}(z_0)) + \mathbb{KL}(q_{\phi}(z_{>0}\vert X) \vert\vert p_{\theta_z}(z_{>0}))
\end{align}
where the first $\mathbb{KL}$ on the r.h.s is a classic between two probability distributions over a random variable, and the second is derived from the Girsanov's theorem as:
\begin{align}
\mathbb{KL}(q_{\phi}(z_{>0}\vert X) \vert\vert p_{\theta_z}(z_{>0})) &= \frac{1}{2} \mathbb{E}_{q_{\phi}(z_{>0}\vert X)} \left( \int_{0}^{T} \vert \Delta(t) \vert^2 dt \right) \\
\Delta(t) &= \sigma_{\theta}^{-1}(z_t,t) (f_{\phi}(z_t, t, \textbf{X}) - f_{\theta}(z_t, t))
\end{align}

Finally:
\begin{align}
\mathcal{L}(\theta, \phi, \textbf{X}) &= \mathbb{E}_{q_{\phi}(z \vert X)} \log{p(x_{t_1:t_N} \vert z_{t_1:t_N})} - \mathbb{KL}(q_{\phi}(z_0\vert X) \vert\vert p_{\theta_z}(z_0)) - \frac{1}{2} \mathbb{E}_{q_{\phi}(z_{>0}\vert X)} \left( \int_{0}^{T} \vert \Delta(t) \vert^2 dt \right)
\end{align}

During training:
- the integral is approximated via numerical integration
- expectations are estimated with MC sampling
- NB : sampling is actually : sampling $z_0 \sim q_{\phi}(z_0 \vert \textbf{X})$ and sampling a function $z$ by sampling a Brownian motion path $B_t$ and computing the whole realization path $z_t$.

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Set Up
</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torchsde
from torchdiffeq import odeint, odeint_adjoint

# from mpl_toolkits.mplot3d import Axes3D
import timeit

In [2]:
def seed_everything(seed=42):
    """
    Set seed for reproducibility.
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')
    dtype = torch.FloatTensor

print(f"Using {device}")

torch.set_default_dtype(torch.float32)

if device.type == 'cuda':
    print('GPU Name:', torch.cuda.get_device_name(0))
    print('Total GPU Memory:', round(torch.cuda.get_device_properties(0).total_memory/1024**3,1), 'GB')

Using cpu


/home/benjamin/anaconda3/envs/torchy/lib/python3.13/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Data and Problem Statement
</div>

- The experiment is an unknown stochastic process $dX_t = f(X_t,t)dt + \sigma(X_t,t)dB_t$
- We observe several sample paths of this stochastic process.
- Each sample path is a collection $\{ (t_1, x_{t_1}), (t_2, x_{t_2}), ..., (t_n, x_{t_n}) \}$ of $n$ data points at observation times $(t_i)_{1 \leq i \leq n}$. NB : $n$ may vary by sample.

In [ ]:
B = 32 # batch size, ie number of sample paths
LENGTH = 100 # total number of points used to draw a full path of the stochastic process
N_POINTS = 50 # total number of points that are actually observed in the stochastic process

Data Generating SDE

In [ ]:
# here, we create an Ornstein Uhlenbeck SDE model:
# dX_t = theta * (mu - X_t) dt + sigma dB_t
# theta, mu, sigma scalar parameters
# B_t is a standard 1D Brownian motion

# SDE are instantiated as subclasses of nn.Module

class DataGeneratingSDE(nn.Module):
    def __init__(self, theta, mu, sigma):
        
        # noise type can take 4 values : "diagonal", "general", "additive", "scalar"
        # here we use "diagonal" : the diffusion function g(t,y) is an element wise function,
        # its output has the same shape as y, ie (batch_size, state_size)
        
        # sde_type can be "ito" or "stratonovich"
        # we use "ito" here. The available methods for computation are Euler(-Maruyama), Milstein, SRK.
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        
        # we register the parameters so we can save them. But we will not train them.
        self.register_buffer("theta", torch.tensor(theta))
        self.register_buffer("mu", torch.tensor(mu))
        self.register_buffer("sigma", torch.tensor(sigma))

    # DRIFT FUNCTION
    # inputs are:
    # - t : a tensor of shape (1,) representing the time stamps
    # - y : a tensor of shape (batch_size, state_size) representing the current state
    # outputs:
    # - a tensor of shape (batch_size, state_size) representing the drift at time t and state y
    # note : the functions f and g must be able to handle inputs of shape (batch_size, state_size)
    # for any batch_size >= 1
    def f(self, t, y):
        return self.theta * (self.mu - y)
    
    # DIFFUSION FUNCTION
    # inputs are:
    # - t : a tensor of shape (1,) representing the time stamps
    # - y : a tensor of shape (batch_size, state_size) representing the current state
    # outputs:
    # - a tensor of shape (batch_size, state_size) representing the diffusion at time t and state y
    # (NB : generally, the output of g is of shape (batch_size, state_size, brownian_size) when noise_type is "general")
    def g(self, t, y):
        return self.sigma * torch.ones_like(y)
    
    def __repr__(self):
        msg = f"Ornstein Uhlenbeck SDE Model - mu = {self.mu:.3f}, theta = {self.theta:.3f}, sigma = {self.sigma:.3f}"
        return msg

In [ ]:
# instantiate model
theta = torch.distributions.Uniform(0.1, 1.0).rsample()
mu = torch.distributions.Uniform(-1.0, 1.0).rsample()
sigma = torch.distributions.Uniform(0.01, 1.0).rsample()
datamodel = DataGeneratingSDE(theta=theta.item(), mu=mu.item(), sigma=sigma.item()).to(device)

print(f"Data Generating Model : {datamodel}")

In [ ]:
# form dataset

# times
t_start = 0.0
t_end = 10.0
times = torch.linspace(t_start, t_end, LENGTH).to(device) # (LENGTH)
all_times = times.repeat(B,1) # (B, LENGTH)
       
print(f"All times for data generation : {all_times.shape} (batch, length)")

In [ ]:
# data
Dx = 1 # dimension - 1 as it is a univariate time serie

# sampling full data paths
y_start = 0.0
y0s = torch.full((B,1),y_start).to(device)

with torch.no_grad():
    all_data = torchsde.sdeint(datamodel, y0s, times, method='euler', dt=1e-3) # (N_POINTS,B,1)
    
all_data = all_data.permute(1,0,2) # (B,N_POINTS,1)
# print(f"Computed full sample paths : {all_data.shape} (length,batch,1)")

# extract some subsets in each sample path
for b in range(B):
    idx = np.random.choice(np.arange(LENGTH), N_POINTS, replace=False)  # indices to pick
    observation_ts = times[idx] # (N_POINTS)
    sampled_d = all_data[b,idx] # (N_POINTS,1)
    if b==0:
        sampled_data = sampled_d.detach().unsqueeze(0)
        observation_times = observation_ts.detach().unsqueeze(0)
    else:
        sampled_data = torch.cat([sampled_data, sampled_d.detach().unsqueeze(0)], dim=0)
        observation_times = torch.cat([observation_times, observation_ts.detach().unsqueeze(0)], dim=0) # (B,N_POINTS)

# print(f"Observations times : {observation_times.shape}")
# print(f"Sampled data : {sampled_data.shape}")

In [ ]:
N_DISPLAY=5
fig, ax = plt.subplots(nrows=1, ncols=N_DISPLAY, figsize=(6*N_DISPLAY,4))
idx = np.random.choice(np.arange(B),N_DISPLAY,replace=False)
for i,id in enumerate(idx):
    ax[i].plot(all_times[id].squeeze().detach().cpu().numpy(), all_data[id,:].squeeze().detach().cpu().numpy(), label='full ground truth path', color='blue')
    ax[i].scatter(observation_times[id].squeeze().detach().cpu().numpy(), sampled_data[id].squeeze().detach().cpu().numpy(), s=100, label='sampled points in path', color='green', marker='o')
    ax[i].legend()
    ax[i].grid()
    ax[i].set_title(f'Sample Path {id}/{B}')
    
fig.suptitle(f'Data samples')
plt.show()

In [ ]:
print(f"DATA GENERATING MODEL")
print(datamodel)
print()
print(f"DATA SUMMARY")
print()
print(f"LENGTH of full sample path : {LENGTH}")
print(f"BATCH of paths : {B}")
print(f"Number of points N_POINTS observed in each path : {N_POINTS}")
print()
print(f"Tensor of full paths : {all_data.shape} (BATCH,LENGTH,1)")
print(f"Tensor of all times : {all_times.shape} (BATCH,LENGTH)")
print()
print(f"Tensor of sampled paths : {sampled_data.shape} (BATCH,N_POINTS,1)")
print(f"Tensor of sampling times : {observation_times.shape} (BATCH,N_POINTS)")

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 38px; color: black; font-weight: bold;">
Model and Pipeline
</div>

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Model : parameters
</div>

In [ ]:
Dx = 1  # observation space dimension
Dz = 4  # latent space dimension
Dl = 16 # LSTM dimension

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Model : posterior 1/2 : Context LSTM
</div>

- Context LSTM - Encodes $(t_i, x_{t_i})_{1 \leq i \leq N}$

In [ ]:
class ContextLSTM(nn.Module):
    """LSTM model class to encode all observations,
    with their observation times, into a context
    vector that will be used in the posterior
    """
    def __init__(self, lstm_dim=Dl, n_layers=1, input_dim=Dx):
        # inputs:
        # lstm_dim : dimension of the LSTM
        # n_layers : number of LSTM layers
        # input_dim : input dimension, defaults to 1 (univariate time series)
        super().__init__()
        self.lstm_dim = lstm_dim
        self.n_layers = n_layers
        self.input_dim = input_dim
        self.lstm = nn.LSTM(
            input_size=self.input_dim + 1, # add 1 for time dimension
            hidden_size=self.lstm_dim,
            num_layers=self.n_layers,
            batch_first=True
        )
        
    def forward(self,t,x):
        # inputs:
        # t : tensor (B,N) of time stamps
        # x : tensor (B,N,input_dim) of observations at times t
        # outputs:
        # h : tensor (B,lstm_dim) of hidden stats encoding (t,x)
        x_ext = torch.cat([t.unsqueeze(-1),x], dim=-1) # (B,N,input_dim+1)
        _,(hn,_) = self.lstm(x_ext) # hn (1,B,lstm_dim)
        
        return hn.squeeze(0) # (B,lstm_dim)

    def __repr__(self):
        description = f"ContextLSTM with lstm_dim = {self.lstm_dim}, n_layers = {self.n_layers}, input_dim = {self.input_dim}"
        description += f"\nLSTM net : {self.lstm}"
        return description

In [ ]:
# test LSTM
context_lstm = ContextLSTM(input_dim=Dx)
print(context_lstm)

t_in = torch.randn(32,50)
x_in = torch.randn(32,50,Dx)

context = context_lstm(t_in, x_in)

print(f"Inputs:")
print(f"\ttimes : {t_in.shape}")
print(f"\tobservations : {x_in.shape}")
print(f"Outputs:")
print(f"\thidden state : {context.shape}")

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Model : posterior 2/2 : Posterior for initial values of the SDEs
</div>

- $q_{\phi}(z_0) \sim \mathcal{N}(\mu_{\phi}(X), \sigma_{\phi}(X))$

In [ ]:
# posterior distribution for starting value

class EncoderStartingValue(nn.Module):
    """
    Encoder for the starting value of the latent variable.
    Uses the context vector from the LSTM to parameterize
    the mean and log-variance of a Gaussian distribution.
    NB : we assume a diagonal covariance matrix
    """
    def __init__(self, context_dim=Dl, latent_dim=Dz, n_layers=1, hidden_dim=32):
        # inputs:
        # context_dim : dimension of the context vector. This is the dimension of the LSTM (Dl)
        # latent_dim ; latent space dimension (Dz)
        # n_layers, hidden_dim : number and size of layers of the MLP
        super().__init__()
        self.context_dim = context_dim
        self.latent_dim = latent_dim
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        layers = []
        input_dim = self.context_dim
        for i in range(self.n_layers):
            layers.append(nn.Linear(input_dim, self.hidden_dim))
            layers.append(nn.Tanh())
            input_dim = self.hidden_dim
        self.mlp = nn.Sequential(*layers)
        self.mean = nn.Linear(self.hidden_dim, self.latent_dim)
        self.logvar = nn.Linear(self.hidden_dim, self.latent_dim)
        
    def forward(self, c):
        # input:
        # c : context vector, tensor (B,context_dim)
        # output:
        # mean : tensor (B, latent_dim) : mean of the Gaussian
        # logvar : tensor (B, latent_dim) : log var of the Gaussian
        x = self.mlp(c)
        mean = self.mean(x)
        logvar = self.logvar(x)
        
        return mean, logvar

    def __repr__(self):
        description = f"Encoder of Starting Value z0 given X\n"
        description += f"\tInput size (context vector) : {self.context_dim}\n"
        description += f"\tOutput dimension (latent vector dimension) : {self.latent_dim}\n"
        description += f"\tMLP : {self.mlp}\n"
        description += f"\tMean : {self.mean}\n"
        description += f"\tLog var : {self.logvar}"
        return description

In [ ]:
encoder_start = EncoderStartingValue()
print(encoder_start)

In [ ]:
K=3

mean, logvar = encoder_start(context)
cov = torch.diag_embed(0.5 * torch.exp(logvar))

q_phi_z0 = torch.distributions.MultivariateNormal(
    loc=mean,
    covariance_matrix=cov
)

z0s = q_phi_z0.rsample((K,))

print(f"Sampling initial values for the SDEs")
print(f"\tDistribution mean : {mean.shape}")
print(f"\tCovariance matrix : {cov.shape}")
print(f"\tDistribution object : {q_phi_z0}")
print(f"Sampling {K} samples : {z0s.shape}")

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Model : Posterior and Prior SDEs
</div>

- Posterior : $dZ_t^{(\phi)} = f_{\phi}(Z_t,t \vert X)dt + \sigma_{\theta}(Z_t,t)dB_t$
- Prior : $dZ_t^{(\theta)} = f_{\theta}(Z_t,t)dt + \sigma_{\theta}(Z_t,t)dB_t$

Here, we choose to implement both the prior and the posterior in the same class (see torchsde.sdeint documentation):
- this allows to share the same diffusion by design
- this allows a library-based computation of the KL between the two SDEs

In [ ]:
class LatentSDE(nn.Module):
    """
    Latent SDE class
    """
    def __init__(self, input_dim=Dx, latent_dim=Dz, lstm_dim=Dl, hidden_dim=32):
        # we use a diagonal noise type
        # the SDE type is "Ito"
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        self.Dx = input_dim
        self.Dz = latent_dim
        self.Dl = lstm_dim
        self.hidden_dim = hidden_dim
        # Ornstein Uhlenbeck prior parameters
        self.kappa = 1.0
        self.mu = 0.0
        # basic z|x encoder seen above
        self.encoder = ContextLSTM(lstm_dim=self.Dl, input_dim=self.Dx)
        self.context = None # context tensor
        # net for posterior drift
        self.posterior_drift = nn.Sequential(
            nn.Linear(self.Dz+self.Dl, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.Dz)
        )
        # net for shared diagonal diffusion
        self.shared_diffusion = nn.Sequential(
            nn.Linear(self.Dz+self.Dl, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.Dz)
        )
        
    def encode(self,t,x):
        # separate method to encode data:
        # inputs :
        # t : time steps, tensor (B,N)
        # x : observations at times t, tensor (B,N,Dx)
        self.context = self.encoder(t,x) # encode into (B,Dl)

    def f(self,t,z):
        """Posterior drift"""
        # assume self.context has been computed
        # inputs:
        # t : times tensor (B,1) : unused. Time is taken into account in the context
        #     computed from self.encoder
        # z : variable tensor (B,Dz)
        # outputs:
        # f(t,z) : drift tensor (B,Dz)
        z_ext = torch.cat([self.context, z], dim=-1) # form (B,Dl+Dz)
        posterior_drift = self.posterior_drift(z_ext) # (B,Dz)
        return posterior_drift

    def g(self,t,z):
        """Shared diffusion"""
        # assume self.context has been computed
        # inputs : t,z as above
        # outputs : g(t,z) : diagonal diffusion (B,Dz)
        z_ext = torch.cat([self.context, z], dim=-1) # form (B,Dl+Dz)
        shared_diffusion = self.shared_diffusion(z_ext) # (B,Dz)
        return shared_diffusion
    
    def h(self,t,z):
        """Prior drift"""
        # here, we posit a fixed Ornstein Uhlenbeck
        prior_drift = self.kappa * (self.mu - z)
        return prior_drift
    
    def __repr__(self):
        d = f"Latent SDE Model - Dz = {self.Dz} - LSTM dim = {self.Dl}\n"
        d += f"\tLSTM Encoder : {self.encoder}\n"
        d += f"\tPosterior Drift : {self.posterior_drift}\n"
        d += f"\tPrior Drift : fixed, O.U with kappa = {self.kappa}; mu = {self.mu}\n"
        d += f"\tShared Diffusion : {self.shared_diffusion}"
        return d

In [ ]:
model_sde = LatentSDE()
print(model_sde)

In [ ]:
# test encode method
print(f"Context tensor before encoding : {model_sde.context}")
model_sde.encode(observation_times, sampled_data)
print(f"Context tensor after encoding : {model_sde.context}")

In [ ]:
observation_times

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Model : prior
</div>

The latent space has dimension $D_z$. 

The latent prior is a Ornstein Uhlenbeck process $\mathbf{Z}$ defined by:
\begin{align*}
z_0^{(\theta)} &\sim p_{\theta_z}(z_0) \\
dz_t^{(\theta)} &= \kappa (\mu - z_t) dt + \sigma_{\theta}(z_t,t)dB_t 
\end{align*}
with: 
\begin{align}
\textbf{drift} \,& f_{\theta}(z_t,t) = \kappa (\mu - z_t) \\
\kappa &\in \mathbb{R} \\
\mu &\in \mathbb{R}^{D_z} \\
\textbf{diffusion} \,& \sigma_{\theta} : \mathbb{R}^{D_z} \times [0,1] \rightarrow \mathbb{R}^{D_z \times D_z} \\
\textbf{Brownian motion} \,& dB_t \in \mathbb{R}^{D_z}
\end{align}

The $\textbf{diffusion}$ is a MLP shared with the posterior

In [ ]:
# Dimension of the latent space
Dz = 4

In [ ]:
class DiagonalSharedDiffusion(nn.Module):
    """The diffusion is shared between the prior and the posterior.
    So we can actually compute a KL
    """
    def __init__(self,n_layers=1, n_hidden=32, latent_dim=Dz):
        super().__init__()
        
        self.register_buffer("n_layers", torch.tensor(n_layers))
        self.register_buffer("n_hidden", torch.tensor(n_hidden))
        self.register_buffer("latent_dim", torch.tensor(latent_dim))
        
        layers = []
        input_dim = self.latent_dim
        for _ in range(self.n_layers):
            layers.append(nn.Linear(input_dim, self.n_hidden))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(self.n_hidden, self.latent_dim ))

        self.mlp = nn.Sequential(*layers)
        
    def forward(self,t,z):
        # inputs:
        # t : tensor (B,N) : NOT USED HERE - ie assume constant diffusion
        # z : tensor (B,Dz)
        # outputs:
        # diff : tensor (B,Dz) > 0
        return torch.exp(self.mlp(z))
    
    def __repr__(self):
        description = f'Model Diagonal Shared Diffusion'
        description += f"{self.mlp}"
        return description
    
shared_diffusion = DiagonalSharedDiffusion().to(device)
print(shared_diffusion)

print(f"Computing diffusion")
z = torch.randn((B,LENGTH,Dz)).to(device)
print(f"Input : {z.shape} (batch,length,Dz)")
diff = shared_diffusion(times, z)
print(f"Output : diffusion = {diff.shape} (batch,length,Dz)")

In [ ]:
class PriorSDE(nn.Module):
    """class for the latent Prior
    This is a O.U. process with kappa=0.5, mu=0.0
    """
    def __init__(self, shared_diffusion, kappa=0.5, mu=0.0):
        super().__init__()
        # we assume a diagonal shared diffusion
        # shared_diffusion is a nn.Module
        # FROM THE DOC : "diagonal": The diffusion g is element-wise and has output size (batch_size, state_size). 
        # The Brownian motion is a batch of state_size-dimensional Brownian motions.
        self.noise_type = 'diagonal'
        self.sde_type = 'ito'
        self.register_buffer("kappa", torch.tensor(kappa))
        self.register_buffer("mu", torch.tensor(mu))
        self.diffusion = shared_diffusion
        
    # DRIFT FUNCTION
    # inputs are:
    # - t : a tensor of shape (1,) representing the time stamps
    # - y : a tensor of shape (batch_size, state_size) representing the current state
    # outputs:
    # - a tensor of shape (batch_size, state_size) representing the drift at time t and state y
    # note : the functions f and g must be able to handle inputs of shape (batch_size, state_size)
    # for any batch_size >= 1
    def f(self,t,z):
        return self.kappa * (self.mu - z)
    
    # DIFFUSION FUNCTION
    # inputs are:
    # - t : a tensor of shape (1,) representing the time stamps
    # - y : a tensor of shape (batch_size, state_size) representing the current state
    # outputs:
    # - a tensor of shape (batch_size, state_size) representing the diffusion at time t and state y
    # (NB : generally, the output of g is of shape (batch_size, state_size, brownian_size) when noise_type is "general")
    def g(self,t,z):
        # we assume shared_diffusion : state_size --> state_size
        return self.diffusion(t,z) * torch.ones_like(z)
    
    def __repr__(self):
        description = f"Model Prior SDE - Ornstein Uhlenbeck"
        description += f"kappa = {self.kappa:.3f}, mu = {self.mu:.3f}"
        description += f"Diffusion : {self.diffusion}"
        return description
        
prior = PriorSDE(shared_diffusion=shared_diffusion).to(device)
print(prior)

In [ ]:
# prior initial values
p_z0_prior = torch.distributions.MultivariateNormal(
    loc = torch.zeros(Dz),
    covariance_matrix = torch.eye(Dz)
)

print(f"Prior initial value z0 : {p_z0_prior}")
print(f"\tBatch shape : {p_z0_prior.batch_shape}")
print(f"\tEvent shape : {p_z0_prior.event_shape}")

# sample a batch of initial values
z0s = p_z0_prior.sample((B,)).to(device)
print(f"Sampling initial values of O.U. priors : {z0s.shape}")

In [ ]:
# sample priors
N = 5
idx = np.random.choice(B,N)
z_priors = torchsde.sdeint(prior, z0s, times, method='euler')

fig, ax = plt.subplots(nrows=N, ncols=Dz, figsize=(6*N,4*Dz))
times_ = times.detach().cpu().numpy()
for i, id in enumerate(idx):
    z_priors_ = z_priors[:,id,:].squeeze().detach().cpu().numpy()  # LENGTH,1,Dz => LENGTH,Dz
    for j in range(Dz):
        ax[i,j].plot(times_, z_priors_[:,j])
        ax[i,j].set_title(f"Sample {id} - component {j}")
        ax[i,j].grid()
        
fig.suptitle(f"Prior samples - {N} samples in dimension {Dz}")
plt.show()

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Creating a SDE model, sampling paths
</div>

In [ ]:
N_POINTS = 50 # number of points in each path
N_PATHS = 10  # number of paths to sample

t_start = 0.0
t_end = 10.0

ts = torch.linspace(t_start, t_end, N_POINTS).to(device)  # time stamps where we want the solution, between 0 and 1
print(f"Time stamps shape : {ts.shape}")  # shape (N_POINTS,)
y_start = 0.25  # initial condition
y0 = torch.full((N_PATHS, 1), y_start).to(device)  # initial condition 0.25, shape (batch_size=N_PATHS, state_size=1)
print(f"Initial condition shape : {y0.shape}")  # shape (N_PATHS, 1)

# now, we call sdeint to solve the SDE
# NB : we can use the adjoint method by calling sdeint_adjoint instead of sdeint
# method can be "euler", "milstein", "srk" for sde_type="ito"
# dt is the step size used by the solver (smaller dt -> more accurate but slower). By default, dt=1e-3

with torch.no_grad():  # we don't need gradients for this demo
    ys = torchsde.sdeint(model, y0, ts, method="euler", dt=1e-3)  # shape (N_POINTS, N_PATHS, 1)
print(f"Computed solution samples : {ys.size()}")

In [ ]:
def plot_ousde_samples(ts, ys, model, title=None):
    """
    Utility functions to plot the sampled SDE solutions
    """
    fig, ax = plt.subplots(figsize=(10,6))
    n_points = ts.size()[0]
    n_paths = ys.size()[1]
    
    for i, y in enumerate(ys.permute(1,0,2)):  # iterate over paths
        ax.plot(ts.detach().cpu().numpy(), y.detach().cpu().numpy(), lw=1, alpha=1.0, label=f'Path {i+1}' if i<10 else None)  # plot each path
    
    if title is None:
        title = f"DATA : Sampled paths of the Ornstein-Uhlenbeck SDE - parameters : mu = {model.mu.item():.1f}, theta = {model.theta.item():.1f}, sigma = {model.sigma.item():.1f}"
    ax.set_title(title)
    ax.set_xlabel("Time")
    ax.set_ylabel("X(t)")
    ax.legend()
    ax.grid()
        
    return fig, ax

fig, ax = plot_ousde_samples(ts, ys, model)
plt.show()

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Toy 1 : learning a O.U. model from the data
</div>

NB - yes this is an ugly way to learn parameters of an O.U. process. It is just to demonstrate a basic use of the torchsde SDE model class.

In [ ]:
class Toy_OU_SDE(nn.Module):
    def __init__(self):
        # we keep "diagonal" noise type
        
        # sde_type can be "ito" or "stratonovich"
        # we use "ito" here. The available methods for computation are Euler(-Maruyama), Milstein, SRK.
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        
        # here, we want to learn the parameters theta, mu, sigma
        self.theta = nn.Parameter(torch.tensor(1.0))  # initial guess
        self.mu = nn.Parameter(torch.tensor(1.0))     # initial guess
        self.sigma = nn.Parameter(torch.tensor(1.0))  # initial guess

    # DRIFT FUNCTION - same signature as above
    def f(self, t, y):
        return self.theta * (self.mu - y)
    
    # DIFFUSION FUNCTION - same signature as above
    def g(self, t, y):
        return self.sigma**2 * torch.ones_like(y)

# instantiate model
model_ou_1 = Toy_OU_SDE().to(device)

In [ ]:
preds = torchsde.sdeint(model_ou_1, y0, ts, method="euler", dt=1e-3)  # shape (N_POINTS, N_PATHS, 1)
print(f"Computed solution samples : {preds.size()}")

fig, ax = plot_ousde_samples(ts, preds, model_ou_1)
plt.show()

In [ ]:
def plot_comparison(learned_model, t_start=t_start, t_end=t_end, N_SAMPLES=5, device=device, gt_model=model, y_start=0.1):
    """
    Utility function to plot comparison between ground truth model and learned model
    """

    bm = torchsde.BrownianInterval(t_start, t_end, size=(N_SAMPLES, 1), device=device) # instantiate N_SAMPLES independent Brownian motions of dimension 1
    bm_increments = torch.stack([bm(t0,t1) for t0,t1 in zip(ts[:-1], ts[1:])])  # sample N_POINTS-1 increments for each BM : shape (N_POINTS-1, N_SAMPLES, 1)
    bm_queries = torch.cat([torch.zeros(1, N_SAMPLES, 1, device=device), torch.cumsum(bm_increments, dim=0)])  # add independent increments to sample Brownian motions : shape (N_POINTS, N_SAMPLES, 1)

    y0s = torch.full((N_SAMPLES, 1), y_start).to(device)  # initial condition 0.1, shape (batch_size=N_SAMPLES, state_size=1)

    with torch.no_grad():
        ys_true = torchsde.sdeint(gt_model, y0s, ts, method="euler", dt=1e-3, bm=bm)  # shape (N_POINTS, N_SAMPLES, 1)
        ys_learned = torchsde.sdeint(learned_model, y0s, ts, method="euler", dt=1e-3, bm=bm)  # shape (N_POINTS, N_SAMPLES, 1)

    fig, ax = plt.subplots(nrows=1, ncols=N_SAMPLES, figsize=(6*N_SAMPLES,6))

    for i in range(N_SAMPLES):
        ax[i].plot(ts.detach().cpu().numpy(), ys_true[:,i,:].detach().cpu().numpy(), lw=2, alpha=1.0, label='Ground truth', color='blue')
        ax[i].plot(ts.detach().cpu().numpy(), ys_learned[:,i,:].detach().cpu().numpy(), lw=2, alpha=1.0, label='Model', color='orange')
        ax[i].set_title(f"Sampled paths - Sample {i+1}")
        ax[i].set_xlabel("Time")
        ax[i].set_ylabel("X(t)")
        ax[i].legend()
        ax[i].grid()
        
    return fig, ax

In [ ]:
# avant training
fig, ax = plot_comparison(model_ou_1)
fig.suptitle("Comparison of the ground truth and O.U. SDE models before training", fontsize=16)
plt.show()

In [ ]:
# training loop parameters
learning_rate = 1e-2

optimizer = torch.optim.Adam(
    list(model_ou_1.parameters()), 
    lr=learning_rate
)

# -------------------------------------------------------
# --- N EPOCHS for all training loops
# -------------------------------------------------------
N_EPOCHS = 200
# -------------------------------------------------------
# -------------------------------------------------------

thetas = []
mus = []
sigmas = []
losses = []

for epoch in range(N_EPOCHS):
    # compute SDE and sample N_PATHS
    preds = torchsde.sdeint(model_ou_1, y0, ts, method="euler", dt=1e-3)  # shape (N_POINTS, N_PATHS, 1)
    # compute average L2 loss between data paths and sampled paths
    mse = (data - preds.permute(1,0,2))**2
    loss = mse.mean()
    # train
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # record and report out
    losses.append(loss.item())
    m = model_ou_1.mu.item()
    s = torch.sqrt(model_ou_1.sigma).item()
    t = model_ou_1.theta.item()
    mus.append(m)
    sigmas.append(s)
    thetas.append(t)
    print(f"Epoch {epoch+1:<3} - {N_EPOCHS:<3} - loss = {loss.item():.3e} - parametres : theta = {t:.3f}, mu = {m:.3f}, sigma = {s:.3f}")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(24,4))

axs[0].plot(losses, label='losses')
axs[1].plot(mus, label='mu')
axs[2].plot(thetas, label='theta')
axs[3].plot(sigmas, label='sigma')

for i in range(4):
    axs[i].set_xlabel('epoch')
    axs[i].legend()
    axs[i].grid()

plt.show()

In [ ]:
# now, we test the two models by sampling the SAME brownian motion for both
# after training
fig, ax = plot_comparison(model_ou_1)
fig.suptitle("Comparison of the ground truth and learned O.U. SDE models after training", fontsize=16)
plt.show()

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Toy 2 : learning a O.U. model from the data with bayesian inference
</div>

Here, we put a Gaussian prior on the O.U. paramters

In [ ]:
class BayesianToy_OU_SDE(nn.Module):
    def __init__(self):
        
        # we keep "diagonal" noise type
        
        # sde_type can be "ito" or "stratonovich"
        # we use "ito" here. The available methods for computation are Euler(-Maruyama), Milstein, SRK.
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        
        # here, we want to learn the parameters theta, mu, sigma
        # we put a prior on them
        self.theta_mean = nn.Parameter(torch.tensor(1.0))  # initial guess
        self.theta_std = nn.Parameter(torch.tensor(0.1))   # initial guess
        self.mu_mean = nn.Parameter(torch.tensor(1.0))     # initial guess
        self.mu_std = nn.Parameter(torch.tensor(0.1))      # initial guess
        self.sigma_mean = nn.Parameter(torch.tensor(1.0))  # initial guess
        self.sigma_std = nn.Parameter(torch.tensor(0.1))   # initial guess

    # DRIFT FUNCTION
    def f(self, t, y):
        # sample parameters from normal distributions
        # NB : need the reparametrization trick to backprop through the sampling
        eps1 = torch.randn_like(self.theta_mean)
        self.theta = self.theta_mean + eps1 * torch.abs(self.theta_std)
        eps2 = torch.randn_like(self.mu_mean)
        self.mu = self.mu_mean + eps2 * torch.abs(self.mu_std)
        
        return self.theta * (self.mu - y)
    
    # DIFFUSION FUNCTION
    def g(self, t, y):
        # sample sigma from normal distribution with reparametrization trick
        eps3 = torch.randn_like(self.sigma_mean)
        self.sigma = self.sigma_mean + eps3 * torch.abs(self.sigma_std)

        return self.sigma**2 * torch.ones_like(y)

# instantiate model
bayesianmodel_ou = BayesianToy_OU_SDE().to(device)

In [ ]:
N_PATHS = 10  # number of paths to sample

y_start = 0.25  # initial condition
y0 = torch.full((N_PATHS, 1), y_start).to(device)  # initial condition 0.25, shape (batch_size=N_PATHS, state_size=1)

preds = torchsde.sdeint(bayesianmodel_ou, y0, ts, method="euler", dt=1e-3)  # shape (N_POINTS, N_PATHS, 1)
print(f"Computed solution samples : {preds.size()}")

In [ ]:
def plot_bayesian_ousde_samples(ts, ys, model):
    """
    Utility functions to plot the sampled SDE solutions
    """
    fig, ax = plt.subplots(figsize=(10,6))
    n_points = ts.size()[0]
    n_paths = ys.size()[1]
    
    for i, y in enumerate(ys.permute(1,0,2)):  # iterate over paths
        ax.plot(ts.detach().cpu().numpy(), y.detach().cpu().numpy(), lw=1, alpha=1.0, label=f'Path {i+1}' if i<10 else None)  # plot each path

    title = f"Sampled paths of the Ornstein-Uhlenbeck SDE - parameters : mu = ({model.mu_mean.item():.1f}, {model.mu_std.item():.1f})" \
        + f", theta = ({model.theta_mean.item():.1f}, {model.theta_std.item():.1f})" \
            + f", sigma = ({model.sigma_mean.item():.1f}, {model.sigma_std.item():.1f})"
    ax.set_title(title)
    ax.set_xlabel("Time")
    ax.set_ylabel("X(t)")
    ax.legend()
    ax.grid()
        
    return fig, ax

fig, ax = plot_bayesian_ousde_samples(ts, preds, bayesianmodel_ou)
plt.show()

In [ ]:
fig, ax = plot_comparison(bayesianmodel_ou)
fig.suptitle("Comparison of the ground truth and learned Bayesian SDE models before training", fontsize=16)
plt.show()

In [ ]:
# training loop parameters
learning_rate = 1e-2

optimizer = torch.optim.Adam(
    list(bayesianmodel_ou.parameters()), 
    lr=learning_rate
)

print(f"Training for {N_EPOCHS} epochs with learning rate {learning_rate}")

losses = []
theta_priors = []
theta_stds = []
mu_priors = []
mu_stds = []
sigma_priors = []
sigma_stds = []

for epoch in range(N_EPOCHS):
    # compute SDE and sample N_PATHS
    preds = torchsde.sdeint(bayesianmodel_ou, y0, ts, method="euler", dt=1e-3)  # shape (N_POINTS, N_PATHS, 1)
    # compute average L2 loss between data paths and sampled paths
    mse = (data - preds.permute(1,0,2))**2
    loss = mse.mean()
    # train
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # record
    losses.append(loss.item())
    mm, ms = bayesianmodel_ou.mu_mean.item(), bayesianmodel_ou.mu_std.item()
    tm, t_s = bayesianmodel_ou.theta_mean.item(), bayesianmodel_ou.theta_std.item()
    sm, ss = bayesianmodel_ou.sigma_mean.item(), bayesianmodel_ou.sigma_std.item()
    mu_priors.append(mm)
    mu_stds.append(ms)
    theta_priors.append(tm)
    theta_stds.append(t_s)
    sigma_priors.append(sm)
    sigma_stds.append(ss)
    # report out
    print(f"Epoch {epoch+1:<3} - {N_EPOCHS:<3} - loss = {loss.item():.3e} - parametres : theta = ({tm:.3e}, {t_s:.3e}), mu = ({mm:.3e}, {ms:.3e}), sigma = ({sm:.3e}, {ss:.3e})")

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,4*4))

axs[0, 0].plot(losses, label='losses')

# Second subplot - mu
axs[1, 0].plot(mu_priors, label='mu mean', color='blue')
axs[1, 1].plot(mu_stds, label='mu std', color='green')
axs[1, 0].set_ylabel('mu mean', color='blue')
axs[1, 1].set_ylabel('mu std', color='green')

# Third subplot - theta
axs[2, 0].plot(theta_priors, label='theta mean', color='blue')
axs[2, 0].set_ylabel('theta mean', color='blue')
axs[2, 1].plot(theta_stds, label='theta std', color='green')
axs[2, 1].set_ylabel('theta std', color='green')

# Fourth subplot - sigma
axs[3, 0].plot(sigma_priors, label='sigma mean', color='blue')
axs[3, 1].plot(sigma_stds, label='sigma std', color='green')
axs[3, 0].set_ylabel('sigma mean', color='blue')
axs[3, 1].set_ylabel('sigma std', color='green')

# Set common properties
for i in range(4):
    axs[i, 0].set_xlabel('epoch')
    axs[i, 1].set_xlabel('epoch')
    axs[i, 0].legend()
    if i > 0: axs[i, 1].legend()
    axs[i, 0].grid()
    axs[i, 1].grid()

plt.show()

In [ ]:
fig, ax = plot_comparison(bayesianmodel_ou)
fig.suptitle("Comparison of the true and learned Bayesian SDE models after training", fontsize=16)
plt.show()

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Toy 3 : learning an arbitrary SDE with a neural SDE model
</div>

Here, we sample paths from a non-trivial stochastic process, and learn drift and diffusion with MLPs

In [ ]:

class DataGeneratingSDE(nn.Module):
    def __init__(self, sigma=0.5):
        
        # we keep "diagonal" noise type
        
        # sde_type can be "ito" or "stratonovich"
        # we use "ito" here. The available methods for computation are Euler(-Maruyama), Milstein, SRK.
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        self.register_buffer("sigma", torch.tensor(sigma))
        
    # DRIFT FUNCTION
    # inputs are:
    # - t : a tensor of shape (1,) representing the time stamps
    # - y : a tensor of shape (batch_size, state_size) representing the current state
    # outputs:
    # - a tensor of shape (batch_size, state_size) representing the drift at time t and state y
    # note : the functions f and g must be able to handle inputs of shape (batch_size, state_size)
    # for any batch_size >= 1
    def f(self, t, y):
        return torch.sin(y)
    
    # DIFFUSION FUNCTION
    # inputs are:
    # - t : a tensor of shape (1,) representing the time stamps
    # - y : a tensor of shape (batch_size, state_size) representing the current state
    # outputs:
    # - a tensor of shape (batch_size, state_size) representing the diffusion at time t and state y
    # (NB : generally, the output of g is of shape (batch_size, state_size, brownian_size) when noise_type is "general")
    def g(self,t,y):
        return self.sigma * torch.ones_like(y)

# datagen_model = DataGeneratingSDE(sigma=0.1).to(device)
datagen_model = OUSDE(theta=2.0, mu=0.5, sigma=0.05)

In [ ]:
# we will generate N_PATHS of the SDE, each computed on N_POINTS
N_PATHS = 10
N_POINTS = 50

# time interval
t_start = 0.0
t_end = 10.0
ts = torch.linspace(t_start, t_end, N_POINTS).to(device)

# initial condition
y_start = 0.50
y0s = torch.full((N_PATHS,1), y_start).to(device)

# sample paths
with torch.no_grad():
    ys = torchsde.sdeint(datagen_model, y0s, ts, method='euler', dt=1e-3)
    
# report out
print(f"Computing {N_PATHS} paths of the SDE, on {N_POINTS} points of the time interval ({t_start},{t_end})")
print(f"Time tensor : {ts.shape}")
print(f"Initial condition tensor : {y0s.shape}")
print(f"Paths samples tensor : {ys.shape}")

In [ ]:
fig, ax = plot_ousde_samples(ts, ys, datagen_model, title='Ground truth SDE samples')
plt.show()

In [ ]:
data = ys.detach()
print(f"Data : {data.shape} - N_POINTS x N_PATHS x Dx")

In [ ]:
# SDE Model to learn
class LearningSDE(nn.Module):
    def __init__(self, input_dim=1, output_dim=1, hidden_dim=256):
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        
        # create 2 MLPs for the drift and the diffusion
        self.drift = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, output_dim)
        )
        self.diffusion = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, output_dim),
            torch.nn.Softplus()
        )
        
    def f(self, t, y):
        return self.drift(y)

    def g(self, t, y):
        return self.diffusion(y)
    
learning_model = LearningSDE().to(device)

In [ ]:
with torch.no_grad():
    outputs = torchsde.sdeint(learning_model, y0s, ts, method='euler', atol=1e-2, rtol=1e-2, dt=1e-3)
    
fig, ax = plot_ousde_samples(ts, outputs, learning_model, title='Learning model before training')
plt.show()

In [ ]:
# Learning Drift and Diffusion networks

# parameters
learning_rate = 1e-3
optimizer = torch.optim.Adam(
    params = list(learning_model.parameters()),
    lr = learning_rate
)
# N_EPOCHS = 200
losses = []
best_loss = torch.tensor(float('inf'))
savefilepath = '/home/benjamin/Folders_Python/MVA_Stage/models/latent_sde/learned_sde.pth'

# reporting
print(f'Starting learning of SDE model on {N_EPOCHS} epochs')

# training loop
learning_model.train()

for epoch in range(N_EPOCHS):
    # compute sample SDE paths from the model
    outputs = torchsde.sdeint(learning_model, y0s, ts, method='euler', atol=1e-5, rtol=1e-5, dt=1e-3) # N_POINTS, N_PATHS, 1
    # compute basic L2 loss between the samples and the data
    mse = (outputs-data)**2
    loss = mse.mean()
    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # record and report out
    losses.append(loss.item())
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(learning_model.state_dict(), savefilepath)
        msg = f'saving model' + ' '*10
    else:
        msg = f'not saving model'
    print(f"Epoch {epoch+1:<3} - {N_EPOCHS:<3} ... loss = {loss.item():.3e} ... best = {best_loss:.3e} ... " + msg, end='\r')

In [ ]:
# restore best model
learning_model = LearningSDE().to(device)
learning_model.load_state_dict(torch.load(savefilepath))

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))

ax.plot(losses, label='training loss')
ax.set_xlabel('Epoch')
ax.set_yscale('log')
ax.set_ylabel('Loss')
ax.legend()
ax.grid()
plt.show()

In [ ]:
# print predictions vs ground truth by setting same Brownian motion samples for both

fig, ax = plot_comparison(learned_model = learning_model, N_SAMPLES=3, gt_model=datagen_model, y_start=y_start)
plt.show()

<div style="background-color: #008B8B; padding: 15px; border-radius: 5px; font-size: 20px; color: black; font-weight: bold;">
Toy 4 : Learning a SDE with a KL on a ground truth SDE
</div>

In [ ]:
class PriorSDE(nn.Module):
    """
    Simple O.U. for prior
    """
    def __init__(self, shared_sigma, theta=1.0, mu=1.0):
        # we keep "diagonal" noise type
        
        # sde_type can be "ito" or "stratonovich"
        # we use "ito" here. The available methods for computation are Euler(-Maruyama), Milstein, SRK.
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        
        # parameters
        self.theta = torch.tensor(theta, requires_grad=True)
        self.mu = torch.tensor(mu, requires_grad=True)
        self.sigma = shared_sigma

    # DRIFT FUNCTION - same signature as above
    def f(self, t, y):
        return self.theta * (self.mu - y)
    
    # DIFFUSION FUNCTION - same signature as above
    def g(self, t, y):
        return self.sigma**2 * torch.ones_like(y)

In [ ]:
class PosteriorSDE(nn.Module):
    """
    Posterior with a neural net
    """
    def __init__(self, shared_sigma, n_layers=1, hidden_dim=32):
        super().__init__()
        self.noise_type = "diagonal"
        self.sde_type = "ito"
        self.register_buffer("n_layers", torch.tensor(n_layers))
        self.register_buffer("hidden_dim", torch.tensor(hidden_dim))
        self.sigma = shared_sigma
        
        layers = []
        input_dim = 1
        for i in range(int(self.n_layers)):
            layers.append(torch.nn.Linear(input_dim, self.hidden_dim))
            layers.append(torch.nn.Tanh())
            input_dim = self.hidden_dim
        layers.append(torch.nn.Linear(self.hidden_dim, 1))
        self.drift = torch.nn.Sequential(*layers)
        
    def f(self, t, y):
        return self.drift(y)
    
    def g(self,t ,y):
        # same diffusion as prior
        return self.sigma**2 * torch.ones_like(y)

In [ ]:
shared_sigma = nn.Parameter(torch.tensor(1.0))
prior = PriorSDE(shared_sigma).to(device)
posterior = PosteriorSDE(shared_sigma).to(device)

In [ ]:
# checking home made integral approximate calculation

# def func(x):
#     # return torch.sin(x)
#     # return x
#     return torch.exp(x)

# def approx_int(f,a,b,n=1000):
#     xs = torch.linspace(a,b,n)  # (n,1)
#     fs = func(xs) # (n, 1)
#     app_int = torch.stack( [1/2 * (xs[i+1]-xs[i]) * (fs[i] + fs[i+1]) for i in range(xs.shape[0]-1) ] ) # (n-1, 1)
#     return app_int.sum()

# a = torch.tensor(0.0)
# b = torch.tensor(1.0)
# # print(f"integral = {torch.cos(a) - torch.cos(b)}")
# # print(f"integral = {1/2*(b**2 - a**2)}")
# print(f"integral = {torch.exp(b) - torch.exp(a)}")
# print(f"approx = {approx_int(func,a,b)}")

In [ ]:
N_POINTS = 50

# time interval
t_start = 0.0
t_end = 10.0
ts = torch.linspace(t_start, t_end, N_POINTS).to(device)

In [ ]:
# Computing KL between prior and posterior

def kl_path (t, prior, posterior, K=3, epsilon=1e-3, device=device):
    """
    compute the KL between the prior and the posterior
    Inputs :
    - t (torch.tensor) : time stamps 
    - prior (nn.Module) : prior with drift and diffusion
    - posterior (nn.Module) : posterior with learnable drift and same diffusion as prior
    - K (int, defaults to 3) : number of samples to estimate the expectation
    Outputs :
    - KL (tensor) : KL between the two stochastic processes
    """
    
    # sample z from posterior
    bm = torchsde.BrownianInterval(t[0], t[-1], size=(K,1), device=device)
    bm_increments = torch.stack([bm(t0,t1) for t0,t1 in zip(t[:-1], t[1:])])
    bm_queries = torch.cat([torch.zeros(1,K, 1, device=device), torch.cumsum(bm_increments, dim=0)]) # n time_steps x K x Dx=1
    
    y_start = 0.0
    y0s = torch.full((K,1), y_start).to(device)
    
    # compute paths
    ys_prior = torchsde.sdeint(prior, y0s, t, method="euler", dt=1e-3, bm=bm)
    ys_posterior = torchsde.sdeint(posterior, y0s, t, method="euler", dt=1e-3, bm=bm)
    # compute drifts
    ys_prior_drift = prior.f(t, ys_prior)
    ys_posterior_drift = posterior.f(t, ys_posterior)
    # compute common diffusion
    diff = posterior.g(t, ys_posterior)
        
    # print(f"prior drift = {ys_prior_drift}")
    # print(f"posterior drift = {ys_posterior_drift}")
    # print(f"diff drift = {diff}")
        
    # compute KL
    
    # make sure we do not divide by something too close to zero
    diff2 = torch.where(diff.abs().detach() > epsilon, diff, torch.full_like(diff, fill_value=epsilon) * diff.sign())
    # print(f"diff stable: {diff2}")
    
    # compute kl
    deltas = torch.div(ys_posterior_drift - ys_prior_drift, diff2)**2
    # delta_t = torch.tensor( [t[i+1]-t[i]] for in range(t.shape[0]-1) )
    approx_int = torch.stack( [1/2 * (t[i+1]-t[i]) * (deltas[i,:,:] + deltas[i+1,:,:]) for i in range(t.shape[0]-1) ] )  # n_steps-1 x K x 1
    # print(f"approx_int {approx_int}")
    kl = torch.sum(approx_int, dim=0).mean()
    # print(kl.shape)
    
    return kl

In [ ]:
kl = kl_path(ts, prior, posterior)
kl.backward()

In [ ]:
# do not pass the shared_sigma parameter twice to the optimizer
prior_params = [p for p in prior.parameters() if p is not shared_sigma]
posterior_params = [p for p in posterior.parameters() if p is not shared_sigma]

In [ ]:
# learning the prior
N_EPOCHS = 100
K = 30
y_start = 0.0
y0s = torch.full((K,1), y_start).to(device)
optimizer = torch.optim.Adam(
    params = prior_params + posterior_params + [shared_sigma],
    lr = 1e-3
)
epsilon = 1e-6
kls = []
sigmas = []

for epoch in range(N_EPOCHS):
    # sample z from posterior
    bm = torchsde.BrownianInterval(ts[0], ts[-1], size=(K,1), device=device)
    bm_increments = torch.stack([bm(t0,t1) for t0,t1 in zip(ts[:-1], ts[1:])])
    bm_queries = torch.cat([torch.zeros(1,K, 1, device=device), torch.cumsum(bm_increments, dim=0)]) # n time_steps x K x Dx=1
    # compute paths, drifts, and common diffusion
    ys_prior = torchsde.sdeint(prior, y0s, ts, method="euler", dt=1e-3, bm=bm)
    ys_posterior = torchsde.sdeint(posterior, y0s, ts, method="euler", dt=1e-3, bm=bm)
    # compute drifts
    ys_prior_drift = prior.f(ts, ys_prior)
    ys_posterior_drift = posterior.f(ts, ys_posterior)
    # compute common diffusion
    diff = posterior.g(ts, ys_posterior)
    # compute kl
    diff2 = torch.where(diff.abs().detach() > epsilon, diff, torch.full_like(diff, fill_value=epsilon) * diff.sign())
    deltas = torch.div(ys_posterior_drift - ys_prior_drift, diff2)**2
    approx_int = torch.stack( [1/2 * (ts[i+1]-ts[i]) * (deltas[i,:,:] + deltas[i+1,:,:]) for i in range(ts.shape[0]-1) ] )  # n_steps-1 x K x 1
    kl = torch.sum(approx_int, dim=0).mean()
    # train
    optimizer.zero_grad()
    kl.backward()
    optimizer.step()
    # reporting
    print(f"Epoch {epoch+1:<3} : {N_EPOCHS:<3} -- kl = {kl.item():.4e} -- diffusion = {shared_sigma.item():.3e}")
    kls.append(kl.item())    
    sigmas.append(posterior.sigma.item())

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,4))
ax[0].plot(kls)
ax[0].grid()
ax[0].set_title(f'KL vs Epoch')
ax[1].plot(sigmas)
ax[1].grid()
ax[1].set_title(f'Shared diffusion')
plt.show()

In [ ]:
# plot samples of both SDEs
N_SAMPLES = 3
bm = torchsde.BrownianInterval(t_start, t_end, size=(N_SAMPLES,1), device=device)
bm_increments = torch.stack([bm(t0,t1) for t0,t1 in zip(ts[:-1],ts[1:])]) # N_POINTS-1, N_SAMPLES, 1
bm_samples = torch.cat( [torch.zeros(1,N_SAMPLES,1,device=device), torch.cumsum(bm_increments,dim=0)]) # N_POINTS, N_SAMPLES_1

y_start = 0.0
y0s = torch.full((N_SAMPLES,1), y_start).to(device) # N_SAMPLES, 1

# compute paths
with torch.no_grad():
    ys_prior = torchsde.sdeint(prior, y0s, ts, method="euler", dt=1e-3, bm=bm)
    ys_posterior = torchsde.sdeint(posterior, y0s, ts, method="euler", dt=1e-3, bm=bm) # N_POINTS, N_SAMPLES, 1
    
fig, ax = plt.subplots(nrows=1, ncols=N_SAMPLES+2, figsize=(8*N_SAMPLES,6))

for i in range(N_SAMPLES):
    # plot trajectories
    ax[i].plot(ts.detach().cpu().numpy(), ys_prior[:,i,:].detach().cpu().numpy(), lw=2, alpha=1.0, label='Prior', color='blue')
    ax[i].plot(ts.detach().cpu().numpy(), ys_posterior[:,i,:].detach().cpu().numpy(), lw=2, alpha=1.0, label='Posterior', color='green')
    ax[i].set_title(f"Sampled paths - Sample {i+1}")
    ax[i].set_xlabel("Time")
    ax[i].set_ylabel("X(t)")
    ax[i].legend()
    ax[i].grid()
    

# compute gradient fields
x_min = torch.tensor(-3.0)
x_max = torch.tensor(+3.0)
N_XS = 10
xs = torch.linspace(x_min, x_max, N_XS).to(device)

T, X = torch.meshgrid(ts, xs, indexing='ij')

ts_flat = T.flatten().unsqueeze(1) # N_POINTS * N_XS, 1
xs_flat = X.flatten().unsqueeze(1) # N_XS * N_POINTS, 1

with torch.no_grad():
    prior_flows = prior.f(ts_flat, xs_flat) # N_XS * N_POINTS, 1
    posterior_flows = posterior.f(ts_flat, xs_flat) # N_XS * N_POINTS, 1
    
# reshape to grid
prior_flows = prior_flows.reshape(T.shape) # N_POINTS, N_XS
posterior_flows = posterior_flows.reshape(T.shape) # N_POINTS, N_XS

# plot
x_points = X.detach().cpu().numpy()
t_points = T.detach().cpu().numpy()
us_prior = prior_flows.detach().cpu().numpy()
us_posterior = posterior_flows.detach().cpu().numpy()

ax[N_SAMPLES].quiver(t_points, x_points, np.ones_like(us_prior), us_prior, scale=20, width=0.003, color='blue', alpha=0.7, label='prior drift')
ax[N_SAMPLES+1].quiver(t_points, x_points, np.ones_like(us_posterior), us_posterior, scale=20, width=0.003, color='green', alpha=0.7, label='posterior drift')
# ax[N_SAMPLES].legend()
ax[N_SAMPLES].grid()
ax[N_SAMPLES+1].grid()
ax[N_SAMPLES].set_title('prior drift')
ax[N_SAMPLES+1].set_title('posterior drift')
    
plt.show()